In [23]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os

# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

# Import constants.py and necessary functions from bit and web3
from web3 import Web3
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI


In [24]:
# Define function to derive wallet
def derive_wallets(mnemonic, coin, numderive):
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [25]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {"eth", "btc-test","btc"}

In [26]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(mnemonic, coin, numderive=3)

In [27]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "19GWVSdQ75pSr8es7PyUM7JcyirrTWYH3v",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "L2JtcN7WUpJbL7SwChvdZz8nKe8hRpNyNX55Q8eYURYCmpuAyLz7",
            "pubkey": "028c8e76eb2a38aae4cd86c96ef7e06fc9ec0bf6c62ffe0a6a045623e3e39ee816",
            "pubkeyhash": "5aafdcef2245eee1a029876e8e0c977c69effc13",
            "xprv": "xprvA3ZJcnNicRrGTMxFZWpCyfNaeKx2xTYWNQy1LJofwFUX5vEvAzAVT6TrwtXtNj291gRHmHRTduNV4YCmzCuwGeNVKPYh1qK3Z57oWQbTAHX",
            "xpub": "xpub6GYf2HucSoQZfr2ifYMDLoKKCMnXMvGMjdtc8hDHVb1Vxia4iXUjztnLo9UgJF6id2P3HhbYmnYMS68ZcNYzoo9Uq1h4FngLtrw4S8pYZBx"
        },
        {
            "address": "1KPfodTztV8W6rVtvHDzEWe4AKD6vWThiJ",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L5ksYi78DASTe9afkZU8cexG1GzCWkyK6ZZtXfvKZWLZ58aMcfzu",
            "pubkey": "02ea0dd72ac9f30f2978521b8e08b488cf81a6cf8fd2371ab56a74e97a121e4e8f",
            "pubkeyhash":

In [33]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [34]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin,account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [35]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_txn(coin,account,recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_btctest = create_tx(coin, account, recipient, amount)
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

In [50]:
# create BTC transaction
create_tx(BTCTEST,btc_acc,"mwYa4qRqPGoMGrD1BjN6GTsp3tsK2bPAPy", 0.000002941)

'{"unspents":[{"amount":72941,"confirmations":0,"script":"76a914afd04a1fc0a0c700948edd984e160536910bcb3a88ac","txid":"cb33366e8d57a38a2f7f7e88f17dac76de73927c2f542a0a302fa7ca5cc18a45","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mwYa4qRqPGoMGrD1BjN6GTsp3tsK2bPAPy",294],["mwYa4qRqPGoMGrD1BjN6GTsp3tsK2bPAPy",49595]]}'

In [51]:
# Send BTC transaction
send_txn(BTCTEST,btc_acc,"mm4tWbiwyyG6LjWPUSA39tQW5kP8EDe8dz", 0.000000041)

0100000001458ac15ccaa72f300a2a542f7c9273de76ac7df1887e7f2f8aa3578d6e3633cb000000006b483045022100eefd5376cb0d0c2693440b5976d044d5ba1995c41ebbe37d1f16f43c88c27b8a022054bc7d29c9aa7cea6eb4c4c23f714f4d261d7826df516cdd38f161db09e2b8d2012102939078ad766bb3be071bb48798b3bf6dacd6576f180a9bc088c8323d7b02e9a7ffffffff0204000000000000001976a9143ce2c7fe432033c4b0ecb4de113e62c1bf4ee1dd88acddc20000000000001976a914afd04a1fc0a0c700948edd984e160536910bcb3a88ac00000000


In [55]:
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [64]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0x74d19dbe60244fa4c80Ff223455eCf8db3993141"))

In [65]:
w3.eth.getBalance("0x74d19dbe60244fa4c80Ff223455eCf8db3993141")

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8545): Max retries exceeded with url: /0x74d19dbe60244fa4c80Ff223455eCf8db3993141 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x00000180214DB0A0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
create_tx(ETH,eth_acc,"0x3dA0F09EF6F833b179768a63AdD29061e0702e47", 1000)

In [ ]:
send_txn(ETH, eth_acc,"0x3dA0F09EF6F833b179768a63AdD29061e0702e47", 1000)